In [1]:
# numpy downgrade가 필요할 수 있음
!python -m pip install numpy==1.2

ERROR: Could not find a version that satisfies the requirement numpy==1.2
ERROR: No matching distribution found for numpy==1.2


In [2]:
!pip install ensemble_boxes

     |████████████████████████████████| 3.4 MB 22.6 MB/s 
  Using cached numpy-1.20.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.3 MB)
     |████████████████████████████████| 26.3 MB 87.9 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.2
    Uninstalling numpy-1.21.2:
      Successfully uninstalled numpy-1.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires numpy~=1.19.2, but you have numpy 1.20.3 which is incompatible.
tensorflow 2.6.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.


In [1]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [2]:
# ensemble csv files
# submission_files = ['./건우님_Yolov5.csv','./범수님_Universe.csv','./준혁님_SwinS.csv','./희수님_Pvtb2.csv','./희수님_Universe.csv','./submission_ensemble_Full_v1_nms.csv','./submission_ensemble_Full_v2.csv','./건우님0.574.csv','./범수님0.624.csv'] # submission lists
submission_files = ['./건우님_Yolov5.csv','./범수님_Universe.csv','./준혁님_SwinS.csv','./희수님_Pvtb2.csv','./희수님_Universe.csv','./건우님0.574.csv','./범수님0.624.csv','./submission_ensemble_nms_1013.csv','./submission_ensemble_WBF_1013.csv'] # submission lists
submission_df = [pd.read_csv(file) for file in submission_files]

In [3]:
submission_files

['./건우님_Yolov5.csv',
 './범수님_Universe.csv',
 './준혁님_SwinS.csv',
 './희수님_Pvtb2.csv',
 './희수님_Universe.csv',
 './건우님0.574.csv',
 './범수님0.624.csv',
 './submission_ensemble_nms_1013.csv',
 './submission_ensemble_WBF_1013.csv']

In [4]:
submission_df

[                                       PredictionString       image_id
 0     7 0.9814453125 602.5 518.0 956.5 1024.0 7 0.97...  test/0000.jpg
 1     5 0.97705078125 345.25 249.75 754.0 693.0 5 0....  test/0001.jpg
 2     1 0.96240234375 77.25 271.5 426.0 604.5 1 0.95...  test/0002.jpg
 3     9 0.95361328125 154.25 257.25 908.0 821.0 5 0....  test/0003.jpg
 4     0 0.8583984375 425.5 409.5 656.5 578.0 0 0.744...  test/0004.jpg
 ...                                                 ...            ...
 4866  1 0.97412109375 293.5 347.0 670.0 688.0 0 0.00...  test/4866.jpg
 4867  3 0.9697265625 416.25 314.25 784.0 706.0 1 0.0...  test/4867.jpg
 4868  7 0.95263671875 119.0 461.5 755.0 838.5 9 0.00...  test/4868.jpg
 4869  0 0.9462890625 0.0 242.5 1024.0 936.5 1 0.7983...  test/4869.jpg
 4870  4 0.9736328125 374.75 201.25 627.0 842.0 7 0.0...  test/4870.jpg
 
 [4871 rows x 2 columns],
                                        PredictionString       image_id
 0     0 0.27425313 491.84195 595.04

In [5]:
image_ids = submission_df[0]['image_id'].tolist()

In [6]:
image_ids

['test/0000.jpg',
 'test/0001.jpg',
 'test/0002.jpg',
 'test/0003.jpg',
 'test/0004.jpg',
 'test/0005.jpg',
 'test/0006.jpg',
 'test/0007.jpg',
 'test/0008.jpg',
 'test/0009.jpg',
 'test/0010.jpg',
 'test/0011.jpg',
 'test/0012.jpg',
 'test/0013.jpg',
 'test/0014.jpg',
 'test/0015.jpg',
 'test/0016.jpg',
 'test/0017.jpg',
 'test/0018.jpg',
 'test/0019.jpg',
 'test/0020.jpg',
 'test/0021.jpg',
 'test/0022.jpg',
 'test/0023.jpg',
 'test/0024.jpg',
 'test/0025.jpg',
 'test/0026.jpg',
 'test/0027.jpg',
 'test/0028.jpg',
 'test/0029.jpg',
 'test/0030.jpg',
 'test/0031.jpg',
 'test/0032.jpg',
 'test/0033.jpg',
 'test/0034.jpg',
 'test/0035.jpg',
 'test/0036.jpg',
 'test/0037.jpg',
 'test/0038.jpg',
 'test/0039.jpg',
 'test/0040.jpg',
 'test/0041.jpg',
 'test/0042.jpg',
 'test/0043.jpg',
 'test/0044.jpg',
 'test/0045.jpg',
 'test/0046.jpg',
 'test/0047.jpg',
 'test/0048.jpg',
 'test/0049.jpg',
 'test/0050.jpg',
 'test/0051.jpg',
 'test/0052.jpg',
 'test/0053.jpg',
 'test/0054.jpg',
 'test/005

In [7]:
annotation = '/opt/ml/detection/dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [8]:
prediction_strings = []
file_names = []
iou_thr = 0.4
skip_box_thr = 0.01

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:85: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:73: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')


In [9]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('submission_ensemble1013_nms.csv',index=False)

submission.head()

,PredictionString,image_id
0,7 0.9586443901062012 602.4054565429688 517.370...,test/0000.jpg
1,6 0.44835874 764.2208251953125 662.87609863281...,test/0001.jpg
2,4 0.8386178612709045 867.0836791992188 244.586...,test/0002.jpg
3,9 0.48192817 142.2207489013672 243.35012817382...,test/0003.jpg
4,1 0.33104828 203.7183837890625 284.93591308593...,test/0004.jpg


In [13]:
df = pd.read_csv("submission_ensemble_Full_v2.csv")
df2 = pd.read_csv('submission_ensemble_Full_v1_nms.csv')

In [14]:
df.drop(columns='Unnamed: 0',inplace=True)

In [16]:
df.to_csv('submission_ensemble_Full_v2.csv',index=False)
df

,PredictionString,image_id
0,7 0.9678258895874023 215.40846252441406 50.373...,test/0000.jpg
1,6 0.5236840844154358 763.5913696289062 662.352...,test/0001.jpg
2,4 0.8348793387413025 866.9296875 244.525085449...,test/0002.jpg
3,9 0.6531789898872375 129.14100646972656 214.07...,test/0003.jpg
4,1 0.41406381130218506 205.48472595214844 282.7...,test/0004.jpg
...,...,...
4866,0 0.5629467964172363 290.96954345703125 346.65...,test/4866.jpg
4867,3 0.8634536266326904 417.3979187011719 316.972...,test/4867.jpg
4868,7 0.5154743194580078 124.11984252929688 458.14...,test/4868.jpg
4869,0 0.8583388328552246 2.126479148864746 236.675...,test/4869.jpg
